In [225]:
import xgboost as xgb
import pandas as pd 
import numpy as np

from sklearn.utils.extmath import softmax

In [226]:
edge_train = pd.read_csv('train_2.csv')
edge_test = pd.read_csv('test_2.csv')
u_ex = pd.read_csv('upload_2.csv')
str_fea = pd.read_csv('con_100fea2.csv')

In [227]:
def edge_sort_train(train):
    train_link = train.loc[:,['to', 'from', 'label']]

    for i in train_link.index:
        t = train_link.loc[i]['to']
        f = train_link.loc[i]['from']
        if t > f :
            train_link.loc[i]['to'] = f
            train_link.loc[i]['from'] = t

    train_link = train_link.sort_values(by='to')
    train_link = train_link[~train_link.duplicated()] # -> with duplicated edge exists
    train_link.index = range(train_link.shape[0])

    return train_link

In [228]:
def edge_sort_test(train):
    train_link = train.loc[:,['id', 'to', 'from']]

    for i in train_link.index:
        t = train_link.loc[i]['to']
        f = train_link.loc[i]['from']
        if t > f :
            train_link.loc[i]['to'] = f
            train_link.loc[i]['from'] = t

    train_link = train_link.sort_values(by='to')
    train_link.index = range(train_link.shape[0])

    return train_link

In [229]:
# np.concatenate((t, f), axis=0)

In [230]:
t = str_fea.loc[edge_train.loc[i, 'to'], :].values
f = str_fea.loc[edge_train.loc[i, 'from'], :].values

In [231]:
(t * f).shape

(100,)

In [232]:
def get_fea(e_sort, str_fea):
    
    fea = []
    for i in e_sort.index:

        t = str_fea.loc[e_sort.loc[i, 'to'], :].values
        f = str_fea.loc[e_sort.loc[i, 'from'], :].values
        fea.append(np.dot(t,f))

    fea = np.array(fea)
    
    return fea

In [233]:
e_sort_train = edge_sort_train(edge_train)
e_sort_test = edge_sort_test(edge_test)
y_train = e_sort_train.label.values


<ipython-input-228-2e4738421c7f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_link.loc[i]['to'] = f
<ipython-input-228-2e4738421c7f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_link.loc[i]['from'] = t


In [234]:
x_train = get_fea(e_sort_train, str_fea)
x_test = get_fea(e_sort_test, str_fea)

In [145]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(x_train, y_train)


RandomForestClassifier(random_state=0)

In [235]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier()
ridge.fit(x_train.reshape(-1, 1), y_train)


RidgeClassifier()

In [214]:
ridge.predict(x_train.reshape(-1, 1))

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [185]:
import lightgbm as lgbm

lgb = lgbm.LGBMClassifier(random_state=0)
lgb.fit(x_train, y_train)


LGBMClassifier(random_state=0)

In [147]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)

C:\Users\bbb50\anaconda3\envs\graph\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

(1886,)

In [240]:
d = ridge.decision_function(x_test.reshape(-1, 1))
d_2d = np.c_[-d, d]
d = {'id': e_sort_test.id.values, 'prob': softmax(d_2d)[:, 1]}
pred = pd.DataFrame(d)

In [246]:
 softmax(d_2d)[:, 1].shape

(1886,)

In [186]:
d = {'id': e_sort_test.id.values, 'prob': lgb.predict_proba(x_test)[:,1]}
pred = pd.DataFrame(d)


In [241]:
for i in u_ex.index:
    u_ex.loc[i, 'prob'] = pred[pred.id == u_ex.loc[i, 'id']].prob.values

In [242]:
u_ex.to_csv('u2.csv', index= False)